## Table of content

1. [Importing relevent libraries](#01)

2. [Loading Dataset](#02)

3. [Distributions](#03)
    - [Age Distribution](#3.1)
    - [Ethnicity Distribution](#3.2)
    - [Gender Distribution](#3.3)
    - [Sample Images](#3.4)

4. [Helper Functions and Classes](#04)
    - [For GPU utilization](#4.1)
    - [For splitting dataset](#4.2)
    - [For image classification](#4.3)
    - [For training and validation](#4.4)
    - [For evaluation metrics](#4.5)

5. [Model for Gender Prediction](#05)
    - [Create TensorDataset](#5.1)
    - [Split dataset and create dataloader](#5.2)
    - [Build and train model](#5.3)
    - [Evaluating training history](#5.4)
    - [Performance on test data](#5.5)
    - [Saving model](#5.6)

6. [Model for Ethnicity Prediction](#06)
    - [Create TensorDataset](#6.1)
    - [Split dataset and create dataloader](#6.2)
    - [Build and train model](#6.3)
    - [Evaluating training history](#6.4)
    - [Performance on test data](#6.5)
    - [Saving model](#6.6)

7. [Model for Age Prediction](#07)
    - [Create TensorDataset](#7.1)
    - [Split dataset and create dataloader](#7.2)
    - [Build and train model](#7.3)
    - [Evaluating training history](#7.4)
    - [Performance on test data](#7.5)
    - [Saving model](#7.6)

8. [Future Work](#08)
  

## Importing relevent libraries <a id="01"></a>

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
import plotly.figure_factory as ff

## Loading Dataset <a id="02"></a>

In [ ]:
df = pd.read_csv('../input/age-gender-and-ethnicity-face-data-csv/age_gender.csv')
ethnicity_list = ["White", "Black", "Asian", "Indian", "Hispanic"]
gender_list = ['Male', 'Female']
age_list = ['Baby','Child','Adult','Elderly']
df.drop (columns={'img_name'},inplace=True)
df.head()

In [ ]:
print('Total rows: {}'.format(df.shape[0]))
print('Total columns: {}'.format(df.shape[1]))

Let's look at element from the dataframe. the Pixel column consist of pixel values as strings. In order to apply any deep learning Algorithm we need to first convert these pixel values to a desirable format i.e. each image in dataset to be of the shape `(1, 48, 48)` 

In [ ]:
# Converting pixels into numpy array
df['pixels']=df['pixels'].apply(lambda x:  np.array(x.split(), dtype="float32"))

# normalizing pixels data
df['pixels'] = df['pixels']/255

X = np.array(df['pixels'].tolist())
X = X.reshape(-1,1,48,48)
X.shape

From the code above you can see that the bins are:

- 0 to 2 = ‘Toddler/Baby’
- 3 to 17 = ‘Child’
- 18 to 65 = ‘Adult’
- above 65 =’Elderly’


In [ ]:
df['age_bins'] = pd.cut(df['age'],bins=[0,2,17,65,150],labels=range(4))

In [ ]:
df.head()

## Distributions <a id="03"></a>

In [ ]:
# calculating distributions
age_dist = df['age_bins'].value_counts()
age_dist.index = age_list
ethnicity_dist = df['ethnicity'].value_counts()
ethnicity_dist.index = ethnicity_list
gender_dist = df['gender'].value_counts()
gender_dist.index = gender_list

### Age Distribtion <a id="3.1"></a>

In [ ]:
px.bar(x=age_dist.index, y=age_dist.values, title='Age Distribution', labels={'x':'age','y':'count'}, width=800, height=400)

### Ethnicity Distribution <a id="3.2"></a>

In [ ]:
px.bar(x=ethnicity_dist.index, y=ethnicity_dist.values, title='Ethnicity Distribution', labels={'x':'Ethnicity','y':'count'}, width=800, height=400)

### Gender Distribution <a id="3.3"></a>

In [ ]:
px.bar(x=gender_dist.index, y=gender_dist.values, title='Gender Distribution', labels={'x':'Gender','y':'count'}, width=800, height=400)

### Sample Images <a id="3.4"></a>

In [ ]:
plt.figure(figsize=(14,14))
for i in range(3720,3736):
  plt.subplot(4,4,(i%16)+1)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(X[i].squeeze(),cmap='gray')
  plt.title(ethnicity_list[df['ethnicity'].iloc[i]] + " " + gender_list[df['gender'].iloc[i]] + " Age:" + str(df['age'].iloc[i]))
  

## Helper Functions and Classes <a id="04"></a>

### For GPU utilization <a id="4.1"></a>


To seamlessly use a GPU, if one is available, we define a couple of helper functions (get_default_device & to_device) and a helper class DeviceDataLoader to move our model & data to the GPU as required.

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)


Based on where you're running this notebook, your default device could be a CPU (torch.device('cpu')) or a GPU (torch.device('cuda'))

In [ ]:
device = get_default_device()
device

### For splitting dataset <a id="4.2"></a>


While building real world machine learning models, it is quite common to split the dataset into 3 parts:

1. **Training set** - used to train the model i.e. compute the loss and adjust the weights of the model using gradient descent.
2. **Validation set** - used to evaluate the model while training, adjust hyperparameters (learning rate etc.) and pick the best version of the model.
3. **Test set** - used to compare different models, or different types of modeling approaches, and report the final accuracy of the model.

In [ ]:
random_seed = 42
torch.manual_seed(random_seed)
def train_validation_test(dataset, val_size: int = 0, test_size: int = 0):
  train_size = len(dataset) - val_size - test_size 
  if (test_size == 0 or val_size == 0 ):
    return random_split(dataset, [train_size, (val_size + test_size)])
  else:
    return random_split(dataset, [train_size, val_size, test_size])

### For image classification <a id="4.3"></a>

In [ ]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

### For training and validation <a id="4.4"></a>

We'll define two functions: `fit` and `evaluate` to train the model using gradient descent and evaluate its performance on the validation set.

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

def plot_accuracies(history):
  accuracies = [x['val_acc'] for x in history]
  fig = px.line(x = range(len(accuracies)), y = accuracies, title='Accuracy vs. No. of epochs', labels={'x': 'Epoch', 'y':'Accuracy'},width=800, height=400)
  fig.show()

def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = list(range(len(train_losses))), y = train_losses,
                        mode='lines+markers',
                        name='Training'))
    fig.add_trace(go.Scatter(x = list(range(len(val_losses))), y = val_losses,
                        mode='lines+markers',
                        name='Validation'))
    fig.update_layout(title='Loss vs. No. of epochs',
                   xaxis_title='Epoch',
                   yaxis_title='Loss', width=800, height=400)
    fig.show()

# Helper function which returns the predicted label for a single image tensor
def predict_label(img, model, label_classes):
  # Convert to a batch of 1
  xb = to_device(img.unsqueeze(0), device)
  # Get predictions from model
  yb = model(xb)
  # Pick index with highest probability
  _, preds  = torch.max(yb, dim=1)
  # Retrieve the class label
  if label_classes:
    return label_classes[preds[0].item()]
  else: 
    return preds[0].item()


### For evaluation metrics <a id="4.5"></a>

In [ ]:
def draw_confusion_matrix(class_labels,model, batch_loader):
  nb_classes = len(class_labels)

  confusion_matrix = torch.zeros(nb_classes, nb_classes)
  with torch.no_grad():
      for (images, classes) in batch_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, dim=1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
          confusion_matrix[t.long(), p.long()] += 1


  # set up figure 
  fig = ff.create_annotated_heatmap(confusion_matrix.tolist(), x=class_labels, y=class_labels, colorscale='Viridis')


  # add custom xaxis title
  fig.add_annotation(dict(font=dict(color="black",size=14),
                          x=0.5,
                          y=-0.15,
                          showarrow=False,
                          text="Predicted value",
                          xref="paper",
                          yref="paper"))

  # add custom yaxis title
  fig.add_annotation(dict(font=dict(color="black",size=14),
                          x=-0.15,
                          y=0.5,
                          showarrow=False,
                          text="Real value",
                          textangle=-90,
                          xref="paper",
                          yref="paper"))

  # adjust margins to make room for yaxis title
  fig.update_layout(margin=dict(t=50, l=200))

  # add colorbar
  fig['data'][0]['showscale'] = True
  fig.show()

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

def get_metrics(batch_loader, model, average_setting = 'binary'):
  preds=[];truth=[]
  with torch.no_grad():
      for (images, classes) in batch_loader:
        outputs = model(images)
        _, pred = torch.max(outputs, dim=1)
        preds = preds + pred.cpu().tolist()
        truth = truth + classes.cpu().tolist()
  print('F1: {}'.format(f1_score(truth, preds,average=average_setting)))
  print('Precision: {}'.format(precision_score(truth, preds, average=average_setting)))
  print('Recall: {}'.format(recall_score(truth, preds, average=average_setting)))

## Model for Gender Prediction <a id="05"></a>

### Create TensorDataset <a id="5.1"></a>

In [ ]:
X_tensor = torch.from_numpy(X)
y = torch.from_numpy(np.array(df['gender']))
dataset = TensorDataset(X_tensor,y)

The numeric label for each element corresponds to index of the element's label in the list of classes.

In [ ]:
dataset.classes = gender_list
print(dataset.classes)

### Split dataset and create dataloader <a id="5.2"></a>

In [ ]:
# split dataset train/validation/test
train_ds, val_ds, test_ds = train_validation_test(dataset=dataset,val_size=2500,test_size=1000)

batch_size=128
# create batchs using dataloader
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=2, pin_memory=True)

### Build and train model <a id="5.3"></a>

In [ ]:
class GenderModel(ImageClassificationBase):
  def __init__(self):
    super().__init__()
    self.network = nn.Sequential(
        # 1st Convolution Layer
        nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),      # 1 x 48 x 48 -> 16 x 48 x 48
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),                                              # 16 x 48 x 48 -> 16 x 24 x 24
        nn.BatchNorm2d(16),
        nn.Dropout(0.2),

        # 2nd Convolution Layer
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),                # 16 x 24 x 24 -> 32 x 22 x 22
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),                                              # 16 x 24 x 24 -> 32 x 22 x 22
        nn.BatchNorm2d(32),
        nn.Dropout(0.2),

        # 3rd Convolution Layer
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.BatchNorm2d(64),
        nn.Dropout(0.2),

         # 4th Convolution Layer
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.BatchNorm2d(128),
        nn.Dropout(0.2),

        # fully connected layer
        nn.Flatten(),
        nn.Linear(128 * 1 * 1,128),
        nn.Dropout(0.5),
        nn.Linear(128,256),
        nn.Dropout(0.5),
        nn.Linear(256,2)
    )
  def forward(self, images):
    return self.network(images)

We can now wrap our training and validation data loaders using `DeviceDataLoader` for automatically transferring batches of data to the GPU (if available), and use `to_device` to move our model to the GPU (if available).

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
model = to_device(GenderModel(), device)

Before we begin training, let's instantiate the model once again and see how it performs on the validation set with the initial set of parameters.

In [ ]:
history = evaluate(model, val_dl)
print("Untrained Accuracy: ", history['val_acc'])
print("Untrained Loss: ", history['val_loss'])

The initial accuracy is around 50%, which is what one might expect from a randomly intialized model (since it has a 1 in 2 chance of getting a label right by guessing randomly).

We'll use the following *hyperparmeters* (learning rate, no. of epochs, batch_size etc.) to train our model.

In [ ]:
num_epochs = 15
opt_func = torch.optim.Adam
lr = 0.001

In [ ]:
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

### Evaluating training history <a id="5.4"></a>

We can also plot the valdation set accuracies to study how the model improves over time.

In [ ]:
plot_accuracies(history)

We can also plot the training and validation losses to study the trend.

In [ ]:
plot_losses(history)

In [ ]:
get_metrics(train_dl,model)

### Performance on Test Data <a id="5.5"></a>

While we have been tracking the overall accuracy of a model so far, it's also a good idea to look at model's results on some sample images. Let's test out our model with some images from the predefined test dataset.

In [ ]:
index=np.random.randint(0,712,25) # predicted label and their images

plt.figure(figsize=(16,16))

for i in range(len(index)):
    plt.subplot(5,5,(i%25)+1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(test_ds[index[i]][0].squeeze())
    plt.xlabel("Predicted :" + predict_label(test_ds[index[i]][0],model,dataset.classes) 
    )
    
plt.show()

As a final step, let's also look at the overall loss and accuracy of the model on the test set. We expect these values to be similar to those for the validation set. If not, we might need a better validation set that has similar data and distribution as the test set (which often comes from real world data).

In [ ]:
test_dl = DeviceDataLoader(DataLoader(test_ds, batch_size*2), device)
result = evaluate(model, test_dl)
print("Test Accuracy: ",result['val_acc'])
print("Test Loss: ",result['val_loss'])
get_metrics(test_dl,model)

In [ ]:
draw_confusion_matrix(gender_list,model,test_dl)

### Saving model <a id="5.6"></a>

Since we've trained our model for a long time and achieved a resonable accuracy, it would be a good idea to save the weights of the model to disk, so that we can reuse the model later and avoid retraining from scratch.

In [ ]:
folder_path = './'
model_pth = 'GenderModel1.pth'

In [ ]:
torch.save(model.state_dict(), folder_path + model_pth)

The `.state_dict` method returns an `OrderedDict` containing all the weights and bias matrices mapped to the right attributes of the model. To load the model weights, we can redefine the model with the same structure, and use the `.load_state_dict` method.

In [ ]:
model2 = to_device(GenderModel(), device)

In [ ]:
model2.load_state_dict(torch.load(folder_path+ model_pth,map_location=torch.device('cpu')))
result = evaluate(model2, test_dl)
print("Test Accuracy: ",result['val_acc'])
print("Test Loss: ",result['val_loss'])

## Model for Ethnicity Prediction <a id="06"></a>

### Create TensorDataset <a id="6.1"></a>

In [ ]:
X_tensor = torch.from_numpy(X)
y = torch.from_numpy(np.array(df['ethnicity']))
dataset = TensorDataset(X_tensor,y)

The numeric label for each element corresponds to index of the element's label in the list of classes.

In [ ]:
dataset.classes = ethnicity_list
print(dataset.classes)

### Split dataset and create dataloader <a id="6.2"></a>

In [ ]:
# split dataset train/validation/test
train_ds, val_ds, test_ds = train_validation_test(dataset=dataset,val_size=2500,test_size=1000)

batch_size=128
# create batchs using dataloader
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=2, pin_memory=True)

### Build and train model <a id="6.3"></a>

In [ ]:
class EthnicityModel(ImageClassificationBase):
  def __init__(self):
    super().__init__()
    self.network = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.BatchNorm2d(16),
        nn.Dropout(0.2),

        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.BatchNorm2d(32),
        nn.Dropout(0.2),

        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.BatchNorm2d(64),
        nn.Dropout(0.2),

        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.BatchNorm2d(128),
        nn.Dropout(0.2),

        nn.Flatten(),
        nn.Linear(128 * 1 * 1,128),
        nn.Dropout(0.5),
        nn.Linear(128,256),
        nn.Dropout(0.5),
        nn.Linear(256,5)
    )
  def forward(self, images):
    return self.network(images)

We can now wrap our training and validation data loaders using `DeviceDataLoader` for automatically transferring batches of data to the GPU (if available), and use `to_device` to move our model to the GPU (if available).

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
model = to_device(EthnicityModel(), device)
model

Before we begin training, let's instantiate the model once again and see how it performs on the validation set with the initial set of parameters.

In [ ]:
history = evaluate(model, val_dl)
print("Untrained Accuracy: ", history['val_acc'])
print("Untrained Loss: ", history['val_loss'])

The initial accuracy is around 10%, which is what one might expect from a randomly intialized model (since it has a 1 in 10 chance of getting a label right by guessing randomly).

We'll use the following *hyperparmeters* (learning rate, no. of epochs, batch_size etc.) to train our model.

In [ ]:
num_epochs = 20
opt_func = torch.optim.Adam
lr = 0.001

In [ ]:
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

### Evaluating training history <a id="6.4"></a>

We can also plot the valdation set accuracies to study how the model improves over time.

In [ ]:
plot_accuracies(history)

We can also plot the training and validation losses to study the trend.

In [ ]:
plot_losses(history)

In [ ]:
get_metrics(train_dl,model, average_setting='weighted')

### Performance on Test Data <a id="6.5"></a>

While we have been tracking the overall accuracy of a model so far, it's also a good idea to look at model's results on some sample images. Let's test out our model with some images from the predefined test dataset.

In [ ]:
index=np.random.randint(0,712,25) # predicted label and their images

plt.figure(figsize=(16,16))

for i in range(len(index)):
    plt.subplot(5,5,(i%25)+1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(test_ds[index[i]][0].squeeze())
    plt.xlabel("Predicted :" + predict_label(test_ds[index[i]][0],model,dataset.classes) 
    )
    
plt.show()

As a final step, let's also look at the overall loss and accuracy of the model on the test set. We expect these values to be similar to those for the validation set. If not, we might need a better validation set that has similar data and distribution as the test set (which often comes from real world data).

In [ ]:
test_dl = DeviceDataLoader(DataLoader(test_ds, batch_size*2), device)
result = evaluate(model, test_dl)
print("Test Accuracy: ",result['val_acc'])
print("Test Loss: ",result['val_loss'])
get_metrics(test_dl,model,'weighted')

In [ ]:
draw_confusion_matrix(ethnicity_list,model,test_dl)

### Saving model <a id="6.6"></a>

Since we've trained our model for a long time and achieved a resonable accuracy, it would be a good idea to save the weights of the model to disk, so that we can reuse the model later and avoid retraining from scratch.

In [ ]:
folder_path = './'
model_pth = 'EthnicityModel1.pth'
torch.save(model.state_dict(), folder_path + model_pth)

The `.state_dict` method returns an `OrderedDict` containing all the weights and bias matrices mapped to the right attributes of the model. To load the model weights, we can redefine the model with the same structure, and use the `.load_state_dict` method.

In [ ]:
 model2 = to_device(EthnicityModel(), device)

In [ ]:
model2.load_state_dict(torch.load(folder_path+ model_pth))
result = evaluate(model2, test_dl)
print("Test Accuracy: ",result['val_acc'])
print("Test Loss: ",result['val_loss'])

## Model for Age Prediction <a id="07"></a>

### Create TensorDataset <a id="7.1"></a>

In [ ]:
X_tensor = torch.from_numpy(X)
y = torch.from_numpy(np.array(df['age_bins']))
dataset = TensorDataset(X_tensor,y)

The numeric label for each element corresponds to index of the element's label in the list of classes.

In [ ]:
dataset.classes = age_list
print(dataset.classes)

### Split dataset and create dataloader <a id="7.2"></a>

In [ ]:
# split dataset train/validation/test
train_ds, val_ds, test_ds = train_validation_test(dataset=dataset,val_size=1500,test_size=500)

batch_size=128
# create batchs using dataloader
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=2, pin_memory=True)

### Build and train model <a id="7.3"></a>

In [ ]:
class AgeModel(ImageClassificationBase):
  def __init__(self):
    super().__init__()
    self.network = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.BatchNorm2d(16),
        nn.Dropout(0.2),

        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.BatchNorm2d(32),
        nn.Dropout(0.2),

        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.BatchNorm2d(64),
        nn.Dropout(0.2),

        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.BatchNorm2d(128),
        nn.Dropout(0.2),

        nn.Flatten(),
        nn.Linear(128 * 1 * 1,128),
        nn.Dropout(0.5),
        nn.Linear(128,256),
        nn.Dropout(0.5),
        nn.Linear(256,4)
    )
  def forward(self, images):
    return self.network(images)

We can now wrap our training and validation data loaders using `DeviceDataLoader` for automatically transferring batches of data to the GPU (if available), and use `to_device` to move our model to the GPU (if available).

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
model = to_device(AgeModel(), device)
model

Before we begin training, let's instantiate the model once again and see how it performs on the validation set with the initial set of parameters.

In [ ]:
history = evaluate(model, val_dl)
print("Untrained Accuracy: ", history['val_acc'])
print("Untrained Loss: ", history['val_loss'])

The initial accuracy is around 10%, which is what one might expect from a randomly intialized model (since it has a 1 in 10 chance of getting a label right by guessing randomly).

We'll use the following *hyperparmeters* (learning rate, no. of epochs, batch_size etc.) to train our model.

In [ ]:
num_epochs = 20
opt_func = torch.optim.Adam
lr = 0.001

In [ ]:
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

### Evaluating training history <a id="7.4"></a>

We can also plot the valdation set accuracies to study how the model improves over time.

In [ ]:
plot_accuracies(history)

We can also plot the training and validation losses to study the trend.

In [ ]:
plot_losses(history)

In [ ]:
get_metrics(train_dl,model, 'weighted')

### Performance on Test Data <a id="7.5"></a>

While we have been tracking the overall accuracy of a model so far, it's also a good idea to look at model's results on some sample images. Let's test out our model with some images from the predefined test dataset.

In [ ]:
index=np.random.randint(0,500,25) # predicted label and their images

plt.figure(figsize=(16,16))

for i in range(len(index)):
    plt.subplot(5,5,(i%25)+1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(test_ds[index[i]][0].squeeze())
    plt.xlabel("("+age_list[test_ds[index[i]][1].item()]+") Predicted: " + predict_label(test_ds[index[i]][0],model,dataset.classes) 
    )
    
plt.show()

As a final step, let's also look at the overall loss and accuracy of the model on the test set. We expect these values to be similar to those for the validation set. If not, we might need a better validation set that has similar data and distribution as the test set (which often comes from real world data).

In [ ]:
test_dl = DeviceDataLoader(DataLoader(test_ds, batch_size*2), device)
result = evaluate(model, test_dl)
print("Test Accuracy: ",result['val_acc'])
print("Test Loss: ",result['val_loss'])
get_metrics(test_dl,model, 'weighted')

In [ ]:
draw_confusion_matrix(age_list,model,test_dl)

### Saving model <a id="7.6"></a>

Since we've trained our model for a long time and achieved a resonable accuracy, it would be a good idea to save the weights of the model to disk, so that we can reuse the model later and avoid retraining from scratch.

In [ ]:
folder_path = './'
model_pth = 'AgeModel1.pth'
torch.save(model.state_dict(), folder_path + model_pth)

The `.state_dict` method returns an `OrderedDict` containing all the weights and bias matrices mapped to the right attributes of the model. To load the model weights, we can redefine the model with the same structure, and use the `.load_state_dict` method.

In [ ]:
model2 = to_device(AgeModel(), device)

In [ ]:
model2.load_state_dict(torch.load(folder_path+ model_pth))
result = evaluate(model2, test_dl)
print("Test Accuracy: ",result['val_acc'])
print("Test Loss: ",result['val_loss'])

## Further Work <a id="8"></a>

There's a lot of scope to experiment here, and we can use the interactive nature of Jupyter to play around with the various parameters. Here are a few ideas:
* Try chaging the hyperparameters to achieve a higher accuracy within fewer epochs.
* Try adding more convolutional layers, or increasing the number of channels in each convolutional layer

In Future, we can try to combine these three models into a singular model branched such that it's becomes capable of predicting each of these three targets simulatenously. We can also imply transfer learning algorithms for possible improvement. in accuracy.

Another possible addition to this note book would be face detection from images that can then be passed to these trained models for age, gender and ethnicity predictions.

## Acknowledgments <a id="9"></a>
For Creating Helper Functions [PyTorch for Deep Learning - Full Course / Tutorial](https://www.youtube.com/watch?v=GIsg-ZUy0MY&t=24544s&ab_channel=freeCodeCamp.org)